In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import date
from unidecode import unidecode
from tqdm.notebook import tqdm
from glob import glob

BASE = "/media/giani/Gianicosas/Magister/Proyecto/"

# Nuevos atributos

A los anteriores, se agrega sub tipo de institución: 
- CFT: Estatal / Privado
- IP: Presencia nacional / Privado

# Raw data

In [2]:
df = pd.read_csv(f"{BASE}work_data/matriz_desercion_5.csv.gz")
df.head()

/tmp/ipykernel_15886/419383739.py:1: DtypeWarning: Columns (17,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"{BASE}work_data/matriz_desercion_5.csv.gz")


,cat_periodo,codigo_unico,mrun,gen_alu,fec_nac_alu,anio_ing_carr_ori,tipo_inst_1,jornada,dur_estudio_carr,dur_proceso_tit,...,acreditada_inst,desertor_1,region_colegio,tipo_ensenanza_colegio,prom_notas_media,dependencia_colegio,fscu,gratuidad,beca,decil
0,2015,I498S6C132J2V1,37.0,1,197808,2015,Centros de Formación Técnica,Vespertino,5,1.0,...,ACREDITADA,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN
1,2015,I111S14C309J2V1,118.0,1,199102,2015,Institutos Profesionales,Vespertino,5,1.0,...,ACREDITADA,0.0,13,310.0,4.975,3.0,0.0,0.0,0.0,6.0
2,2015,I106S3C64J4V1,253.0,1,198404,2015,Institutos Profesionales,A Distancia,5,1.0,...,NO ACREDITADA,1.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN
3,2015,I221S1C57J1V1,370.0,2,198910,2015,Centros de Formación Técnica,Diurno,5,1.0,...,ACREDITADA,1.0,13,310.0,5.100,3.0,0.0,0.0,1.0,1.0
4,2015,I176S8C59J2V1,533.0,2,198309,2015,Institutos Profesionales,Vespertino,5,1.0,...,NO ACREDITADA,0.0,4,510.0,5.200,2.0,0.0,0.0,0.0,NaN


## Selección periodo
- `cat_periodo` entre 2015 y 2019 inclusive. 

In [3]:
df = df[df["cat_periodo"] < 2020]

# Nulos

In [5]:
df["fec_nac_alu"] = df["fec_nac_alu"].replace([190001], [np.nan])
df["anio_ing_carr_ori"] = df["anio_ing_carr_ori"].replace([9995], [np.nan])
df["acreditada_carr"] = df["acreditada_carr"].str.strip().replace(["", "SIN INFORMACION"], [np.nan, np.nan])
df["acreditada_inst"] = df["acreditada_inst"].str.strip().replace([""], [np.nan])
print(df.shape)
df = df.dropna(subset=[
    "fec_nac_alu", "anio_ing_carr_ori", "acreditada_carr", "acreditada_inst",
    "region_colegio", "tipo_ensenanza_colegio", "prom_notas_media", "dependencia_colegio"
])
print(df.shape)

(1802577, 25)
(1560671, 25)


# Pre procesamiento variables dependientes datos desagregados

## Diferencia año ingreso y último año de matrícula

In [6]:
g = df.groupby(["mrun", "codigo_unico"]).agg({
    "cat_periodo": "max",
    "anio_ing_carr_ori": "min"
}).reset_index()
g["dif_ing_ult_mat"] = g["cat_periodo"] - g["anio_ing_carr_ori"]
g.head()

,mrun,codigo_unico,cat_periodo,anio_ing_carr_ori,dif_ing_ult_mat
0,19.0,I536S0C39J1V1,2019,2019,0
1,118.0,I111S14C309J2V1,2015,2015,0
2,118.0,I221S1C113J2V1,2017,2015,2
3,196.0,I280S8C380J1V1,2019,2017,2
4,238.0,I143S15C614J1V1,2017,2016,1


In [7]:
if "dif_ing_ult_mat" not in df.columns:
    print(df.shape)
    df = pd.merge(
        df,
        g[["mrun", "codigo_unico", "dif_ing_ult_mat"]],
        how="left"
    )
    print(df.shape)

(1560671, 25)
(1560671, 26)


In [8]:
df["dif_ing_ult_mat"].value_counts("%") * 100

2    31.432698
1    28.825102
0    23.873257
3    12.011949
4     3.856995
Name: dif_ing_ult_mat, dtype: float64

## Edad

In [9]:
# Se pisa año de ingreso por el minimo
print(df.shape)
df = pd.merge(
    df.drop(columns=["anio_ing_carr_ori"]),
    g[["mrun", "codigo_unico", "anio_ing_carr_ori"]],
    how="left"
)
df.shape

(1560671, 26)


(1560671, 26)

In [10]:
df["fecha_n"] = pd.to_datetime([
    date(int(str(d)[:4]), int(str(d)[-2:]), 15)
    for d in df["fec_nac_alu"].astype(int)
])

df["Edad"] = [
    date(y, 1, 1).year - fn.year - (
        (date(y, 1, 1).month, date(y, 1, 1).day) < (fn.month, fn.day)
    )
    for fn, y in zip(df["fecha_n"], df["anio_ing_carr_ori"])
]
df.shape

(1560671, 28)

## Valor arancel

In [11]:
valores_uf_dic_anio_anterior = {
    2014: 24627.10,
    2015: 25629.09,
    2016: 26347.98,
    2017: 26798.14,
    2018: 27565.79,
}

In [12]:
df["valor_arancel"] = [
    int(
        valores_uf_dic_anio_anterior[a] * va
    ) if fv == "Monto en UF" else va
    for a, fv, va in zip(
        df["cat_periodo"], df["formato_valores"], df["valor_arancel"]
    )
]

In [13]:
g = df.groupby(["mrun", "codigo_unico"]).agg({
    "valor_arancel": "mean",
}).reset_index()

print(df.shape)
df = pd.merge(
    df.drop(columns=["valor_arancel"]),
    g[["mrun", "codigo_unico", "valor_arancel"]],
    how="left"
)
print(df.shape)

(1560671, 28)
(1560671, 28)


## Decil
- Se usa el valor de decil más reciente

In [26]:
# Se comprueba que hay gente con más de un decil
df.groupby(["mrun"]).agg({"decil":pd.Series.nunique}).reset_index().sort_values(by=["decil"], ascending=False).head()

,mrun,decil
350480,11943353.0,5
63444,2174364.0,5
166909,5692960.0,5
493367,16822245.0,5
591636,20182108.0,5


In [27]:
def fill_decil(data, mrun):
    tmp = data[data["mrun"]==mrun][["cat_periodo", "decil"]].dropna()
    decil = np.nan
    
    if tmp.empty == False:
        decil = tmp.sort_values(
            by=["cat_periodo"], ascending=False
        )["decil"].iloc[0]
        
    return decil

In [3]:
df = pd.read_csv(f"{BASE}tmp_data/decil_imput.csv.gz")

/tmp/ipykernel_29375/1524720311.py:1: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"{BASE}tmp_data/decil_imput.csv.gz")


In [4]:
# Se comprueba que solo hay 1 valor de decil por persona
df.groupby(["mrun"]).agg({"decil":pd.Series.nunique}).reset_index().sort_values(by=["decil"], ascending=False).head()

,mrun,decil
0,19.0,1
464692,15851334.0,1
464717,15852004.0,1
464718,15852090.0,1
464719,15852102.0,1


In [7]:
df["decil"] = df["decil"].astype("category")

# Registros únicos

- Se crea 1 registro por combinación `mrun` - `codigo_unico`.

In [8]:
for c in ["cat_periodo", "anio_ing_carr_ori", "fecha_n", "fec_nac_alu"]:
    if c in df.columns:
        df = df.drop(columns=[c])
        
df.shape

(1560671, 24)

In [9]:
# Se mantiene el registro donde esté como desertor
t = df.drop_duplicates().sort_values(by=["mrun", "codigo_unico", "desertor_1"], ascending=[False, False, False]).drop_duplicates(subset=["mrun", "codigo_unico"], keep="first").copy()
t = t.dropna()
t.shape

(731868, 24)

In [10]:
# No debe haber mrun-codigo_unico repetidos
assert t[["mrun", "codigo_unico"]].value_counts().reset_index()[0].max() == 1

In [11]:
# Se revisa dif
t["dif_ing_ult_mat"].value_counts("%") * 100

0    39.919220
1    28.077058
2    22.406500
3     7.375510
4     2.221712
Name: dif_ing_ult_mat, dtype: float64

In [12]:
print("Desertores")
print(t[t["desertor_1"]==1]["dif_ing_ult_mat"].value_counts("%") * 100)
print("\nNo desetores")
print(t[t["desertor_1"]==0]["dif_ing_ult_mat"].value_counts("%") * 100)

Desertores
0    55.568224
1    25.314826
2    13.241907
3     4.624066
4     1.250977
Name: dif_ing_ult_mat, dtype: float64

No desetores
0    35.675901
1    28.826053
2    24.891533
3     8.121581
4     2.484933
Name: dif_ing_ult_mat, dtype: float64


In [13]:
t["desertor_1"].value_counts("%")

0.0    0.786686
1.0    0.213314
Name: desertor_1, dtype: float64

In [14]:
# Se evalúa decil
t["decil"].value_counts("%") * 100

4.0     47.760252
5.0     13.398454
6.0      9.371362
7.0      6.920510
9.0      6.792482
8.0      5.637492
2.0      2.908038
3.0      2.883990
1.0      2.260790
10.0     2.066630
Name: decil, dtype: float64

# Pre procesamiento resto de variables
## Tipo enseñanza

In [15]:
omit_dep = [
    'LOS ANDES', 'SANTIAGO ORIENTE', 'VALPARAISO', 'SANTIAGO PONIENTE',
    'COLCHAGUA', 'VALPARAÍSO', 'CAUTÍN NORTE', 'ANTOFAGASTA - TOCOPILLA'
]

t = t[~t["dependencia_colegio"].isin(omit_dep)]
t.shape

(731675, 24)

In [16]:
t["dependencia_colegio"] = t["dependencia_colegio"].astype(int)
t["dependencia_colegio"].unique()

array([3, 2, 1, 6, 5, 4])

In [17]:
t["dependencia_colegio"] = t["dependencia_colegio"].map({
    1: 'Municipal',
    2: 'Municipal',
    3: 'Particular Subvencionado',
    4: 'Particular Pagado',
    5: 'Otro',
    6: 'Otro'
})
t["dependencia_colegio"].value_counts()

Particular Subvencionado    373327
Municipal                   299035
Otro                         44421
Particular Pagado            14892
Name: dependencia_colegio, dtype: int64

## Dependencia Colegio

In [18]:
t["tipo_ensenanza_colegio"] = t["tipo_ensenanza_colegio"].map({
    0: "no_aplica",
    10: "Parvularia",
    110: "Basica",
    160: "Basica",
    161: "Basica",
    163: "Carceles",
    165: "Basica",
    167: "Basica",
    211: "Especial",
    212: "Especial",
    213: "Especial",
    214: "Especial",
    215: "Especial",
    216: "Especial",
    217: "Especial",
    299: "PIE",
    310: "HC",
    360: "HC",
    361: "HC",
    362: "Carceles",
    363: "HC",
    410: "TP Comercial",
    460: "TP Comercial",
    461: "TP Comercial",
    463: "TP Comercial",
    510: "TP Industrial",
    560: "TP Industrial",
    561: "TP Industrial",
    563: "TP Industrial",
    610: "TP Tecnica",
    660: "TP Tecnica",
    661: "TP Tecnica",
    663: "TP Tecnica",
    710: "Otro",
    760: "Otro",
    761: "Otro",
    763: "Otro",
    810: "Otro",
    860: "Otro",
    863: "Otro",
    910: "Otro",
    963: "Otro"                     
})

t["tipo_ensenanza_colegio"].value_counts()

HC               402790
TP Comercial     122491
TP Industrial    120993
TP Tecnica        69646
Otro              15755
Name: tipo_ensenanza_colegio, dtype: int64

## Región colegio

In [19]:
t["region_colegio"] = t["region_colegio"].astype(int)
t["region_colegio"].unique()

array([13,  7,  6,  2,  5, 10,  9, 16,  4,  1, 14,  8, 11, 12,  3, 15])

In [20]:
t["region_colegio"] = t["region_colegio"].map({
    1: "Tarapacá",
    2: "Antofagasta",
    3: "Atacama",
    4: "Coquimbo",
    5: "Valparaíso",
    6: "Lib. Gral B. O'Higgins",
    7: "Maule",
    8: "Biobío",
    9: "La Araucanía",
    10: "Los Lagos",
    11: "Aysén",
    12: "Magallanes",
    13: "Metropolitana",
    14: "Los Ríos",
    15: "Arica y Parinacota",
    16: "Ñuble"
})
t["region_colegio"].value_counts()

Metropolitana             292776
Valparaíso                 76656
Biobío                     69731
Maule                      43851
Lib. Gral B. O'Higgins     42315
La Araucanía               37799
Los Lagos                  34057
Coquimbo                   34025
Antofagasta                22674
Ñuble                      17888
Los Ríos                   16208
Arica y Parinacota         11355
Tarapacá                   10754
Atacama                     9956
Magallanes                  7147
Aysén                       4483
Name: region_colegio, dtype: int64

In [21]:
t["misma_region"] = np.where(t["region_colegio"]==t["region_sede"], 1, 0)
t["misma_region"].value_counts("%") * 100

1    88.842041
0    11.157959
Name: misma_region, dtype: float64

## Genero, desertor, beneficios

In [22]:
t["gen_alu"] = t["gen_alu"].replace([1, 2], ["hombre", "mujer"])
t["desertor_1"] = t["desertor_1"].astype(int)
t["fscu"] = t["fscu"].astype(int)
t["gratuidad"] = t["gratuidad"].astype(int)
t["beca"] = t["beca"].astype(int)
t["gen_alu"].value_counts()

mujer     371492
hombre    360183
Name: gen_alu, dtype: int64

## Duraciones

In [23]:
t["dur_estudio_carr"] = t["dur_estudio_carr"].astype(int)
t["dur_proceso_tit"] = t["dur_proceso_tit"].astype(int)

## Instituciones

In [24]:
def iter_df_read(file, col_name, condition_list, chunksize=1000, **kwargs):
    iter_csv = pd.read_csv(file, iterator=True, chunksize=chunksize, **kwargs)
    return pd.concat([chunk[chunk[col_name].isin(condition_list)] for chunk in iter_csv])

In [25]:
cols = ['tipo_inst_1', 'codigo_unico', 'nomb_inst']
tipo_inst = ['Centros de Formación Técnica', 'Institutos Profesionales']

instituciones = pd.concat([
    iter_df_read(f, "tipo_inst_1", tipo_inst, sep=";", usecols=cols)
    for f in tqdm(glob(f"{BASE}/raw_data/post_2015/20220719_Matrícula_Ed_Superior_*.csv"), total=8)
]).drop_duplicates().reset_index(drop=True)

  0%|          | 0/8 [00:00<?, ?it/s]

In [26]:
assert instituciones.groupby(["codigo_unico"]).agg({"nomb_inst": pd.Series.nunique}).reset_index()["nomb_inst"].max() == 1

In [27]:
if "nomb_inst" not in t.columns:     
    t_inicial = t.shape[0]
    t = pd.merge(
        t,
        instituciones,
        how="left"
    )

    assert t.shape[0] == t_inicial
    
    cft_estatal = [
        'CFT DE LA REGION DE ANTOFAGASTA',
        'CFT DE LA REGION DE ARICA Y PARINACOTA',
        'CFT DE LA REGION DE ATACAMA',
        'CFT DE LA REGION DE AYSEN DEL GENERAL CARLOS IBAÑEZ DEL CAMPO',
        'CFT DE LA REGION DE COQUIMBO',
        'CFT DE LA REGION DE LA ARAUCANIA',
        'CFT DE LA REGION DE LOS LAGOS',
        'CFT DE LA REGION DE LOS RIOS',
        'CFT DE LA REGION DE MAGALLANES Y ANTARTICA CHILENA',
        'CFT DE LA REGION DE TARAPACA',
        'CFT DE LA REGION DE VALPARAISO',
        'CFT DE LA REGION DEL BIO BIO',
        "CFT DE LA REGION DEL LIBERTADOR GENERAL BERNARDO O'HIGGINS",
        'CFT DE LA REGION DEL MAULE',
        'CFT DE LA REGION METROPOLITANA DE SANTIAGO'
    ]
    
    ip_nac = [
        'IP AIEP',
        'IP DE CHILE',
        'IP DUOC UC',
        'IP INACAP',
        'IP IPG',
        'IP LATINOAMERICANO DE COMERCIO EXTERIOR - IPLACEX',
        'IP SANTO TOMAS',
    ]

    t["sub_tipo_inst"] = np.where(
        t["nomb_inst"].isin(cft_estatal),
        "cft_estatal",
        np.where(
            t["tipo_inst_1"] == "Centros de Formación Técnica",
            "cft_privado",
            np.where(
                t["nomb_inst"].isin(ip_nac),
                "ip_privado_precencia_nacional",
                "ip_privado"
            )
        )
    )

    assert t.shape[0] == t_inicial
    
t["sub_tipo_inst"].value_counts(dropna=False)

ip_privado_precencia_nacional    414550
cft_privado                      242655
ip_privado                        73627
cft_estatal                         843
Name: sub_tipo_inst, dtype: int64

# Sacar cols auxiliares

In [28]:
t_aux = t.copy().drop(columns=[
    "region_colegio", "nomb_inst", "formato_valores"
]).reset_index(drop=True)

dummy_aux = pd.get_dummies(
    t_aux.drop(columns=["codigo_unico", "mrun"])
).reset_index(drop=True)

dummy_aux = pd.concat([
    t_aux[["mrun", "codigo_unico"]],
    dummy_aux
], axis=1)

dummy_aux.columns = [
    unidecode(c.replace(" ", "_").replace(".", "").replace("'", "").lower())
    for c in dummy_aux.columns
]
dummy_aux.to_csv(f"{BASE}work_data/matriz_desercion_5_dummy_con_id.csv.gz", index=False, compression="gzip")

In [29]:
t = t.drop(columns=["codigo_unico", "mrun", "region_colegio", "nomb_inst", "formato_valores"])

# No dummy

In [30]:
t.head()

,gen_alu,tipo_inst_1,jornada,dur_estudio_carr,dur_proceso_tit,region_sede,area_conocimiento,acreditada_carr,acreditada_inst,desertor_1,...,dependencia_colegio,fscu,gratuidad,beca,decil,dif_ing_ult_mat,Edad,valor_arancel,misma_region,sub_tipo_inst
0,hombre,Institutos Profesionales,Vespertino,5,1,Metropolitana,Tecnología,ACREDITADA,ACREDITADA,1,...,Particular Subvencionado,0,0,1,1.0,0,31,1850000.0,1,ip_privado_precencia_nacional
1,hombre,Centros de Formación Técnica,Vespertino,6,1,Maule,Tecnología,NO ACREDITADA,NO ACREDITADA,0,...,Particular Subvencionado,0,0,0,6.0,0,29,803000.0,1,cft_privado
2,mujer,Institutos Profesionales,Diurno,5,1,Metropolitana,Derecho,NO ACREDITADA,ACREDITADA,0,...,Particular Subvencionado,0,0,0,7.0,0,16,1480000.0,1,ip_privado_precencia_nacional
3,hombre,Institutos Profesionales,Vespertino,8,0,Metropolitana,Administración y Comercio,ACREDITADA,ACREDITADA,1,...,Particular Subvencionado,0,0,0,4.0,0,17,1700900.0,1,ip_privado_precencia_nacional
4,hombre,Institutos Profesionales,Diurno,7,1,Metropolitana,Tecnología,NO ACREDITADA,ACREDITADA,0,...,Municipal,0,0,0,4.0,0,18,1600000.0,1,ip_privado_precencia_nacional


In [31]:
t.to_csv(f"{BASE}work_data/matriz_desercion_5_no_dummy.csv.gz", index=False, compression="gzip")

# Dummys

In [32]:
t.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 731675 entries, 0 to 731674
Data columns (total 22 columns):
 #   Column                  Non-Null Count   Dtype   
---  ------                  --------------   -----   
 0   gen_alu                 731675 non-null  object  
 1   tipo_inst_1             731675 non-null  object  
 2   jornada                 731675 non-null  object  
 3   dur_estudio_carr        731675 non-null  int64   
 4   dur_proceso_tit         731675 non-null  int64   
 5   region_sede             731675 non-null  object  
 6   area_conocimiento       731675 non-null  object  
 7   acreditada_carr         731675 non-null  object  
 8   acreditada_inst         731675 non-null  object  
 9   desertor_1              731675 non-null  int64   
 10  tipo_ensenanza_colegio  731675 non-null  object  
 11  prom_notas_media        731675 non-null  float64 
 12  dependencia_colegio     731675 non-null  object  
 13  fscu                    731675 non-null  int64   
 14  grat

In [33]:
dummy = pd.get_dummies(t, drop_first=True)
dummy.head()

,dur_estudio_carr,dur_proceso_tit,desertor_1,prom_notas_media,fscu,gratuidad,beca,dif_ing_ult_mat,Edad,valor_arancel,...,decil_4.0,decil_5.0,decil_6.0,decil_7.0,decil_8.0,decil_9.0,decil_10.0,sub_tipo_inst_cft_privado,sub_tipo_inst_ip_privado,sub_tipo_inst_ip_privado_precencia_nacional
0,5,1,1,5.5,0,0,1,0,31,1850000.0,...,0,0,0,0,0,0,0,0,0,1
1,6,1,0,4.7,0,0,0,0,29,803000.0,...,0,0,1,0,0,0,0,1,0,0
2,5,1,0,6.4,0,0,0,0,16,1480000.0,...,0,0,0,1,0,0,0,0,0,1
3,8,0,1,5.9,0,0,0,0,17,1700900.0,...,1,0,0,0,0,0,0,0,0,1
4,7,1,0,5.7,0,0,0,0,18,1600000.0,...,1,0,0,0,0,0,0,0,0,1


In [34]:
dummy.columns = [
    unidecode(c.replace(" ", "_").replace(".", "").replace("'", "").lower())
    for c in dummy.columns
]

dummy.columns

Index(['dur_estudio_carr', 'dur_proceso_tit', 'desertor_1', 'prom_notas_media',
       'fscu', 'gratuidad', 'beca', 'dif_ing_ult_mat', 'edad', 'valor_arancel',
       'misma_region', 'gen_alu_mujer', 'tipo_inst_1_institutos_profesionales',
       'jornada_diurno', 'jornada_otro', 'jornada_semipresencial',
       'jornada_vespertino', 'region_sede_arica_y_parinacota',
       'region_sede_atacama', 'region_sede_aysen', 'region_sede_biobio',
       'region_sede_coquimbo', 'region_sede_la_araucania',
       'region_sede_lib_gral_b_ohiggins', 'region_sede_los_lagos',
       'region_sede_los_rios', 'region_sede_magallanes', 'region_sede_maule',
       'region_sede_metropolitana', 'region_sede_tarapaca',
       'region_sede_valparaiso', 'region_sede_nuble',
       'area_conocimiento_agropecuaria',
       'area_conocimiento_arte_y_arquitectura',
       'area_conocimiento_ciencias_basicas',
       'area_conocimiento_ciencias_sociales', 'area_conocimiento_derecho',
       'area_conocimiento_educ

- En género elimina hombre.
- En tipo institución elimina CFT.
- En `region_sede` elimina Antofagasta.
- En `area_conocimiento` elimina Administración y Comercio.
- En ambas acreditaciones elimina Acreditada.
- En `tipo_ensenanza_colegio` elimina HC.
- En `dependencia_colegio` elimina Municipal.

In [35]:
dummy.to_csv(f"{BASE}work_data/matriz_desercion_5_dummy.csv.gz", index=False, compression="gzip")

# Dummy sin drop first

In [36]:
dummy_ohe = pd.get_dummies(t)
dummy_ohe.head()

,dur_estudio_carr,dur_proceso_tit,desertor_1,prom_notas_media,fscu,gratuidad,beca,dif_ing_ult_mat,Edad,valor_arancel,...,decil_5.0,decil_6.0,decil_7.0,decil_8.0,decil_9.0,decil_10.0,sub_tipo_inst_cft_estatal,sub_tipo_inst_cft_privado,sub_tipo_inst_ip_privado,sub_tipo_inst_ip_privado_precencia_nacional
0,5,1,1,5.5,0,0,1,0,31,1850000.0,...,0,0,0,0,0,0,0,0,0,1
1,6,1,0,4.7,0,0,0,0,29,803000.0,...,0,1,0,0,0,0,0,1,0,0
2,5,1,0,6.4,0,0,0,0,16,1480000.0,...,0,0,1,0,0,0,0,0,0,1
3,8,0,1,5.9,0,0,0,0,17,1700900.0,...,0,0,0,0,0,0,0,0,0,1
4,7,1,0,5.7,0,0,0,0,18,1600000.0,...,0,0,0,0,0,0,0,0,0,1


In [37]:
dummy_ohe.columns = [
    unidecode(c.replace(" ", "_").replace(".", "").replace("'", "").lower())
    for c in dummy_ohe.columns
]

In [39]:
dummy_ohe.columns

Index(['dur_estudio_carr', 'dur_proceso_tit', 'desertor_1', 'prom_notas_media',
       'fscu', 'gratuidad', 'beca', 'dif_ing_ult_mat', 'edad', 'valor_arancel',
       'misma_region', 'gen_alu_hombre', 'gen_alu_mujer',
       'tipo_inst_1_centros_de_formacion_tecnica',
       'tipo_inst_1_institutos_profesionales', 'jornada_a_distancia',
       'jornada_diurno', 'jornada_otro', 'jornada_semipresencial',
       'jornada_vespertino', 'region_sede_antofagasta',
       'region_sede_arica_y_parinacota', 'region_sede_atacama',
       'region_sede_aysen', 'region_sede_biobio', 'region_sede_coquimbo',
       'region_sede_la_araucania', 'region_sede_lib_gral_b_ohiggins',
       'region_sede_los_lagos', 'region_sede_los_rios',
       'region_sede_magallanes', 'region_sede_maule',
       'region_sede_metropolitana', 'region_sede_tarapaca',
       'region_sede_valparaiso', 'region_sede_nuble',
       'area_conocimiento_administracion_y_comercio',
       'area_conocimiento_agropecuaria',
       'are

In [38]:
dummy_ohe.to_csv(f"{BASE}work_data/matriz_desercion_5_dummy_ohe.csv.gz", index=False, compression="gzip")